# Final Project Phase 3 Summary
This Jupyter Notebook (.ipynb) will serve as the skeleton file for your submission for Phase 3 of the Final Project. Complete all sections below as specified in the instructions for the project, covering all necessary details. We will use this to grade your individual code (Do this whether you are in a group or not). Good luck! <br><br>

Note: To edit a Markdown cell, double-click on its text.

# Phase II Revisions


Use the following cell (or more if you need them) to make any needed revisions or additions to your Phase II data cleaning. If you were not recommended to make any changes, and don't have any new data sources for Phase III, you may skip this part.


In [ ]:
def data_cleaning():
  pass





############ Function Call ############
data_cleaning()

# Inconsistency Revisions
 **If you were requested to revise your inconsistency section from Phase II, enter your responses here. Otherwise, ignore this section.**

For each inconsistency (NaN, null, duplicate values, empty strings, etc.) you discover in your datasets, write at least 2 sentences stating the significance, how you identified it, and how you handled it.

1. 

2. 

3. 

4. (if applicable)

5. (if applicable)


## Data Sources

Include sources (as links) to your datasets. If any of these are different from your sources used in Phase II, please <b>clearly</b> specify.

*   Downloaded Dataset Source: https://static.stooq.com/db/h/5_us_txt.zip (stooq.com) (Different from phase 1)
*   Web Collection #1 Source: https://www.optionstrategist.com/calculators/free-volatility-data (Same as phase 1)
*   Web Collection #2 Source: "https://alpha-vantage.p.rapidapi.com/query" (Same as phase 1)




# Data Analysis
For the Data Analysis section, you are required to utilize your data to complete the following:

*   Create at least 5 insights
*   Generate at least 3 data visualizations
*   Export aggregated data to at least 1 summary file 

Create a function for each of the following sections mentioned above. Do not forget to fill out the explanation section for each function. 

Make sure your data analysis is not too simple. Performing complex aggregation and using modules not taught in class shows effort, which will increase the chance of receiving full credit. 

# Graphical User Interface (GUI) Implementation
If you decide to create a GUI for Phase II, please create a separate Python file (.py) to build your GUI. You must submit both the completed PhaseIII.ipynb and your Python GUI file.

## Insights

In [475]:
import pandas as pd
import os
import csv
from datetime import datetime
import numpy as np
import re
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from datetime import datetime
from bokeh.models import LinearAxis, Range1d

def insight1(stock):
    directory = r'D:\all_stocks\zzz'
    for entry in os.scandir(directory):
        filename = entry.name
        if filename == stock + ".csv":
            with open(entry.path, "r" ,encoding = "utf-8") as f:
                reader = csv.reader(f, delimiter = ",")
                reader_list = list(reader)
                for item in reader_list[1:]:
                    year = item[2][0:4]
                    month = item[2][4:6]
                    day = item[2][6:8]
                    item[2] = month + "/" + day + "/" + year 
                    
                    item[3] = str(int(item[3])-60000)
                    d = datetime.strptime(item[3], "%H%M%S").time()
                    item[3] = d.strftime("%I:%M %p")
                    
                    item[1] = "5 mins"
                
                    for i in item[1:]:
                        if "." in i:
                            x = item.index(i)
                            item[x] = str('{:.3f}'.format(float(i)))
#                 pprint(reader_list)
                        
                array = np.array(reader_list)
                rawheaders = [reader_list[0]]
                headers = []
                for item in rawheaders[0]:
                    headers.append(re.sub(r"[<>]"," ",item))
                headers[8] = "VOLUME"
                headers[1] = "INTERVAL"
                headers[9] = "INT"
                headers[3] = "TIME (ET)"
                headers[2] = "DATE"
                headers[7] = "CLOSE"
                df = pd.DataFrame(array[1:], columns = headers)
                df.drop(["INT"], axis = 1, inplace = True)
                value = df['VOLUME'].isna().sum()
                df["VOLUME"] = df['VOLUME'].astype(float)
                df["CLOSE"] = df['CLOSE'].astype(float)
# Calculate OBV:
                
                volumeavg = df.groupby(['DATE'],as_index = False)['VOLUME'].mean()
                closeavg = df.groupby(['DATE'],as_index = False)['CLOSE'].mean()
#                 display(newdf)
                
                rawdate = [i for i in df["DATE"]]
                datelist = list(set(rawdate))
                datelist.sort()
                volumelist = [round(i) for i in volumeavg["VOLUME"]]
                closelist = [round(i,2) for i in closeavg["CLOSE"]]
                
                obvlist = []
                obv = 0
                obvlist.append(obv)
                for i in range(1, len(closelist)):
                    if closelist[i-1] < closelist[i]:
                        obv = obvlist[i-1] + volumelist[i]
                        obvlist.append(obv)
                    elif closelist[i-1] > closelist[i]:
                        obv = obvlist[i-1] - volumelist[i]
                        obvlist.append(obv)
                    else:
                        obv = obvlist[i-1]
                        obvlist.append(obv)
                
                price = np.array(obvlist, dtype="float64")
                close = np.array(closelist, dtype="float64")
                timetype = []
                for item in datelist:
                    timetype.append(datetime.strptime(item, "%m/%d/%Y"))
                #     print(timetype)
                odict = {i:j  for i,j in zip(timetype,obvlist)}
                time1 = np.array(timetype, dtype=np.datetime64)
                window_size = 30
                window = np.ones(window_size)/float(window_size)
                stockavg = np.convolve(price, window, 'same')

                p = figure(x_axis_type="datetime", title = stock + " ON-Balance Volume Indicator")
                p.grid.grid_line_alpha = 0.5
                p.xaxis.axis_label = 'Date/Time'
                p.yaxis.axis_label = 'OBV'
                p.ygrid.band_fill_color = "darkorange"
                p.ygrid.band_fill_alpha = 0.4
                p.extra_y_ranges['Price'] = Range1d(start = close.min()-100, end = close.max() +100)
                ax2 = LinearAxis(y_range_name="Price", axis_label="Stock Price")

                p.add_layout(ax2, 'right')

                p.triangle_pin(time1, price, size=10, legend_label='OBV Value',
                       color='blue', alpha=0.3, hatch_color = 'black')
                p.line(time1, price, legend_label='OBV', color='cyan', line_width = 2)
                
                p.scatter(time1, closelist, size=10, legend_label='Close',
                       color='green', alpha=0.3, hatch_color = 'black', y_range_name="Price")
                p.line(time1, closelist, legend_label='Price', color='green', line_width = 4, y_range_name="Price")
                p.legend.location = "top_left"

                show(p)
                return obvlist




############ Function Call ############
insight1("GOOGL")

[0,
 5653,
 10937,
 4894,
 -7417,
 -15700,
 -26862,
 -17720,
 -31660,
 -23600,
 -17526,
 -10969,
 -4081,
 380,
 -10529,
 -4988,
 3671,
 11519,
 16650,
 22620,
 17392,
 11342,
 -2276,
 -11240,
 1384,
 24832,
 15400,
 26860,
 14486,
 23330,
 16405,
 26439,
 37250,
 32257,
 27769,
 20687,
 16517]

### Insight 1 Explanation

Volume of a stock shows the number of trades happening in a fixed, consistent amount of time during the trading day. This shows how many investors and sellers are interested in trading their stock share. There are several indicators that investors use on volume such as the On-Balance Volume (OBV), the volume price trend indicator, the volume relative strength index, etc. Since this insight is about the On-Balance Volume Indicator, On-Balance Volume Indicator (OBV) refers to a technical indicator of momentum that utilizes the positive or negative flow of the volume of trading to reflect the relative buying and selling pressure on a financial asset, with the aim of predicting the probable direction of near-term price changes. In other words, OBV tracks increasing or decreasing volume to project future price movements.

In [474]:
import pandas as pd
import os
import csv
from datetime import datetime
import numpy as np
import re
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from datetime import datetime
from bokeh.models import HoverTool

def insight2(stock):
    directory = r'D:\all_stocks\zzz'
    for entry in os.scandir(directory):
        filename = entry.name
        if filename == stock + ".csv":
            with open(entry.path, "r" ,encoding = "utf-8") as f:
                reader = csv.reader(f, delimiter = ",")
                reader_list = list(reader)
                for item in reader_list[1:]:
                    year = item[2][0:4]
                    month = item[2][4:6]
                    day = item[2][6:8]
                    item[2] = month + "/" + day + "/" + year 
                    
                    item[3] = str(int(item[3])-60000)
                    d = datetime.strptime(item[3], "%H%M%S").time()
                    item[3] = d.strftime("%I:%M %p")
                    
                    item[1] = "5 mins"
                
                    for i in item[1:]:
                        if "." in i:
                            x = item.index(i)
                            item[x] = str('{:.3f}'.format(float(i)))
#                 pprint(reader_list)
                        
                array = np.array(reader_list)
                rawheaders = [reader_list[0]]
                headers = []
                for item in rawheaders[0]:
                    headers.append(re.sub(r"[<>]"," ",item))
                headers[8] = "VOLUME"
                headers[1] = "INTERVAL"
                headers[9] = "INT"
                headers[3] = "TIME (ET)"
                headers[2] = "DATE"
                headers[7] = "CLOSE"
                df = pd.DataFrame(array[1:], columns = headers)
                df.drop(["INT"], axis = 1, inplace = True)
                value = df['VOLUME'].isna().sum()
                df["VOLUME"] = df['VOLUME'].astype(float)
                df["CLOSE"] = df['CLOSE'].astype(float)
#             Calculate VPT
                volumeavg = df.groupby(['DATE'],as_index = False)['VOLUME'].mean()
                closeavg = df.groupby(['DATE'],as_index = False)['CLOSE'].mean()
#                 display(newdf)
                
                rawdate = [i for i in df["DATE"]]
                datelist = list(set(rawdate))
                datelist.sort()
                volumelist = [round(i) for i in volumeavg["VOLUME"]]
                closelist = [round(i,2) for i in closeavg["CLOSE"]]
                
                vptlist = []
                vpt = 0
                vptlist.append(vpt)
                for i in range(1, len(closelist)):
                    vpt = vptlist[i-1] + (volumelist[i] * ((closelist[i]-closelist[i-1])/closelist[i-1]))
                    vptlist.append(round(vpt,2))

                price = np.array(vptlist, dtype="float64")
                close = np.array(closelist, dtype="float64")
                timetype = []
                for item in datelist:
                    timetype.append(datetime.strptime(item, "%m/%d/%Y"))
                #     print(timetype)
                vdict = {i:j for i,j in zip(timetype,vptlist)}
                time1 = np.array(timetype, dtype=np.datetime64)
                window_size = 30
                window = np.ones(window_size)/float(window_size)
                stockavg = np.convolve(price, window, 'same')

                p = figure(x_axis_type="datetime", title = stock + " Volume Price Trend (VPT) Indicator")
                p.grid.grid_line_alpha = 0.5
                p.xaxis.axis_label = 'Date/Time'
                p.yaxis.axis_label = 'VPT'
                p.ygrid.band_fill_color = "olive"
                p.ygrid.band_fill_alpha = 0.3
                p.extra_y_ranges['Price'] = Range1d(start = close.min()-100, end = close.max() +100)
                ax2 = LinearAxis(y_range_name="Price", axis_label="Stock Price")

                p.add_layout(ax2, 'right')

                p.triangle_pin(time1, price, size=10, legend_label='VPT Value',
                       color='blue', alpha=0.3, hatch_color = 'black')
                p.line(time1, price, legend_label='VPT', color='cyan', line_width = 2)
                
                p.scatter(time1, closelist, size=10, legend_label='Close', 
                          color='green', alpha=0.5, hatch_color = 'black', y_range_name="Price")
                
                p.legend.location = "top_left"
            
                show(p)
                return vptlist





############ Function Call ############
insight2("GOOGL")

[0,
 52.04,
 61.03,
 46.78,
 -393.76,
 -453.29,
 -495.67,
 -428.59,
 -746.0,
 -521.76,
 -503.42,
 -361.58,
 -359.31,
 -351.04,
 -617.57,
 -586.22,
 -376.13,
 -348.68,
 -317.62,
 -274.4,
 -308.08,
 -338.65,
 -717.39,
 -736.79,
 -559.26,
 638.29,
 612.65,
 713.06,
 527.93,
 565.08,
 560.49,
 770.15,
 839.53,
 836.69,
 808.0,
 718.96,
 718.7]

### Insight 2 Explanation

This insight uses the Volume Price Trend Indicator or VPT. This indicator is used to determine the balance between the demand and supply of a stock. It can help a trader to relate a stock's price and its trading volume. If volume remains constant, but there is an increase in price, it usually suggests that the price movement is weak, and there is a high chance of it being reversed. A trader who uses VPT and observes the fact, may not pursue long trades, expecting the market to go up further. The VPT is very similiar to OBV, except that in OBV, the increase and decrease in price affects the the indicator but the price is not considered in the calculation, but the VPT takes regard of the percentage change in the stock's price. Indicators are used to see divergence compared to the stock's price graph. If there is a divergence, that means the movement is weak and will reverse.

In [473]:
from ipynb.fs.full.PhaseII import web_parser1
from bokeh.models.glyphs import  Text
from bokeh.palettes import Spectral6
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure
def insight3(stock):
    df = web_parser1("df")
    df["HV-20"] = df['HV-20'].astype(float)
    df["HV-50"] = df['HV-50'].astype(float)
    df["HV-100"] = df['HV-100'].astype(float)
    df["Current IV"] = df['Current IV'].astype(float)
    
    index = df.index
    symbol = df["Symbol"] == stock
    ticker = index[symbol]
    tickerindex = ticker[0]
    hv20 = df.iloc[tickerindex,1]
    hv50 = df.iloc[tickerindex,2]
    hv100 = df.iloc[tickerindex,3]
    iv = df.iloc[tickerindex, 5]
    hvavg = df.iloc[tickerindex,[1,2,3]].mean()

    vt = ['20-day HV','50-day HV','100-day HV','Average HV','IV']
    v = [hv20, hv50, hv100, hvavg, iv]
    vdict = {i:j for i,j in zip(vt,v)}

    source = ColumnDataSource(data=dict(vt=vt, v=v, color=Spectral5))

    p = figure(x_range=vt, y_range=(0,max(v)+10), height=250, title="Volatilities Comparison for " + stock,
               toolbar_location=None, tools="")
    
    p.vbar(x='vt', top='v', width=0.9, color='color', legend_field="vt", source=source)

    p.xgrid.grid_line_color = "olive"
    p.legend.orientation = "horizontal"
    p.legend.location = "top_center"
    tooltips = [
    ("Volatility: ", "@vt"),
    ("Value: ", "@v"),
    ]
    hover = HoverTool(tooltips=tooltips)
    p.add_tools(hover)
    show(p)
    return v
    
    


############ Function Call ############
insight3("GOOGL")

[23.0, 25.0, 22.0, 23.333333333333332, 19.91]

### Insight 3 Explanation 

<b> Hover mouse over bars to get value </b> <br>
Volatility of a stock means the rate at which the price of a stock is increasing or decreasing over a certain amount of time. There are two types of volatilites, Implied Volatility and Historical Volatility. 
Implied Volatility is the market's forecast of a likely movement in a stock's price.
Historical Volatiltiy is the opposite of Implied Volatility (IV) being the record of how the price of a security(stock/option/futures) has fluctuated in the past.

I have collected data on three types of Historical Volatilities 20-day, 50-day, and 100-day historical volatilities, along with current implied volatilties which are refereshed every week. By looking at this representation, you can see if a stock is too volatile, it can quickly become profitable or take you at a loss, since high volatility means more trades happening than usual.

Here, I will intend to show what a divergence in the HV vs IV graph can tell us. For example, if a stock's HV is high but IV is low, this may make a stock an interest for investors.

In [476]:
import requests
from pprint import pprint
import json
import numpy as np
from datetime import datetime

def insight4(stock):
    
    url = "https://alpha-vantage.p.rapidapi.com/query"
    querystring = {"time_period":"30","interval":"5min","series_type":"close","function":"SMA","symbol":stock,"datatype":"json"}
    headers = {
        'x-rapidapi-host': "alpha-vantage.p.rapidapi.com",
        'x-rapidapi-key': "1ae12c90f4msh38344d1088c0185p1a0587jsn0d32c0044762"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    data = response.json()
    
    rawtime = list(data['Technical Analysis: SMA'].keys())
    time = np.array([i[11:] for i in rawtime])

    newtime = [item.replace(":",".") for item in time]
    float_time = np.array(newtime, dtype="float64")    
    
    rawprice = list(data['Technical Analysis: SMA'].values())
    price = np.array([i["SMA"] for i in rawprice], dtype="float64")
    
    adict = {'Time': float_time[:50], 'Price': price[:50]}
    times = list(data['Technical Analysis: SMA'].keys())
    newtimes = []
    for item in times:
        year = item[0:4]
        month = item[5:7]
        day = item[8:10]
        time = item[11:16]
        newtimes.append(str(month + "/" + day + "/" + year + " " + time))

    timetype = []
    for item in newtimes:
        timetype.append(datetime.strptime(item, "%m/%d/%Y %H:%M"))
        
#     bdict = {i:j for i,j in zip(timetype, price)}
    
    WMAlist = []
    WMAlist.append(price[0])
    WMAlist.append(price[1])
    WMAlist.append(price[2])
    WMAlist.append(price[3])

    WMA = round(((price[0]*5+price[1]*4+price[2]*3+price[3]*2+price[4]*1)/15),2)
    WMAlist.append(WMA)
    for i in range(5, len(price)):
        WMA = (price[i-4]*5+price[i-3]*4+price[i-2]*3+price[i-1]*2+price[i]*1)/15
        WMAlist.append(round(WMA,2))
    
    price = np.array(price, dtype="float64")
    close = np.array(WMAlist, dtype="float64")
    time1 = np.array(timetype, dtype=np.datetime64)
    window_size = 30
    window = np.ones(window_size)/float(window_size)
    stockavg = np.convolve(price, window, 'same')
    
    wdict = {i:j for i,j in zip(timetype,WMAlist)}
    p = figure(x_axis_type="datetime", title = stock + " WMA Indicator")
    p.grid.grid_line_alpha = 0.5
    p.xaxis.axis_label = 'Date/Time'
    p.yaxis.axis_label = 'VPT'
    p.ygrid.band_fill_color = "olive"
    p.ygrid.band_fill_alpha = 0.3
    p.extra_y_ranges['WMA'] = Range1d(start = close.min()-100, end = close.max() +100)
    ax2 = LinearAxis(y_range_name="WMA", axis_label="Stock Price")

    p.add_layout(ax2, 'right')

    p.scatter(time1, price, size=10, legend_label='WMA',
           color='blue', alpha=0.1, hatch_color = 'black')
    p.line(time1, price, legend_label='SMA', color='cyan', line_width = 2)

    p.scatter(time1, WMAlist, size=8, legend_label='Close', 
              color='lightgreen', alpha=0.3, hatch_color = 'black', y_range_name="WMA")
    p.line(time1, WMAlist, legend_label='Price', color='green', line_width = 3, y_range_name="WMA")
    p.legend.location = "top_left"


    show(p)
    return WMAlist
#     print(WMAlist, len(WMAlist))
#     print(price, len(price))







############ Function Call ############
insight4("GOOGL")

[2869.1492,
 2868.1802,
 2867.2141,
 2866.2079,
 2867.86,
 2866.9,
 2865.95,
 2865.02,
 2864.12,
 2863.19,
 2862.26,
 2861.34,
 2860.37,
 2859.42,
 2858.47,
 2857.55,
 2856.7,
 2855.91,
 2855.12,
 2854.4,
 2853.73,
 2853.16,
 2852.62,
 2852.04,
 2851.38,
 2850.67,
 2849.93,
 2849.07,
 2848.21,
 2847.34,
 2846.48,
 2845.62,
 2844.78,
 2844.03,
 2843.34,
 2842.61,
 2841.97,
 2841.39,
 2840.84,
 2840.3,
 2839.77,
 2839.27,
 2838.79,
 2838.43,
 2838.11,
 2837.79,
 2837.38,
 2836.95,
 2836.48,
 2836.09,
 2835.97,
 2836.08,
 2836.53,
 2837.34,
 2838.49,
 2839.85,
 2841.34,
 2843.0,
 2844.81,
 2846.89,
 2849.15,
 2851.48,
 2853.89,
 2856.31,
 2858.94,
 2861.7,
 2864.54,
 2867.31,
 2870.06,
 2872.83,
 2875.64,
 2878.41,
 2881.16,
 2883.81,
 2886.43,
 2889.16,
 2892.11,
 2895.16,
 2898.32,
 2901.39,
 2904.22,
 2906.9,
 2909.28,
 2911.36,
 2913.17,
 2914.81,
 2916.34,
 2917.82,
 2919.18,
 2920.29,
 2921.2,
 2922.06,
 2922.81,
 2923.53,
 2923.93,
 2924.2,
 2924.24,
 2924.22,
 2924.21,
 2924.16,
 

### Insight 4 Explanation

A Weighted Moving Average puts more weight on recent data and less on past data. This is done by multiplying each bar’s price by a weighting factor. Because of its unique calculation, WMA will follow prices more closely than a corresponding Simple Moving Average. (Fidelity) If the prices dip around the WMA then it can be a buy signal, and when they rally up towards or close to the WMA it can be an indication to sell. This graph shows the WMA and the SMA. The WMA has a shorter delay then the SMA. WMA can identify trends sooner than a SMA. On the flip side, the WMA will probably experience more whipsaws than a corresponding SMA. (Fidelity)

In [471]:
import math
def insight5(stock):
    
    url = "https://alpha-vantage.p.rapidapi.com/query"
    querystring = {"time_period":"30","interval":"5min","series_type":"close","function":"SMA","symbol":stock,"datatype":"json"}
    headers = {
        'x-rapidapi-host': "alpha-vantage.p.rapidapi.com",
        'x-rapidapi-key': "1ae12c90f4msh38344d1088c0185p1a0587jsn0d32c0044762"
        }
    response = requests.request("GET", url, headers=headers, params=querystring)
    data = response.json()
    
    rawtime = list(data['Technical Analysis: SMA'].keys())
    time = np.array([i[11:] for i in rawtime])

    newtime = [item.replace(":",".") for item in time]
    float_time = np.array(newtime, dtype="float64")    
    
    rawprice = list(data['Technical Analysis: SMA'].values())
    price = np.array([i["SMA"] for i in rawprice], dtype="float64")
    
    adict = {'Time': float_time[:50], 'Price': price[:50]}
    times = list(data['Technical Analysis: SMA'].keys())
    newtimes = []
    for item in times:
        year = item[0:4]
        month = item[5:7]
        day = item[8:10]
        time = item[11:16]
        newtimes.append(str(month + "/" + day + "/" + year + " " + time))

    timetype = []
    for item in newtimes:
        timetype.append(datetime.strptime(item, "%m/%d/%Y %H:%M"))
    
    WMAlist = []
    WMAlist.append(price[0])
    WMAlist.append(price[1])
    WMAlist.append(price[2])
    WMAlist.append(price[3])

    WMA = round(((price[0]*5+price[1]*4+price[2]*3+price[3]*2+price[4]*1)/15),2)
    WMAlist.append(WMA)
    
    for i in range(5, len(price)):
        WMA = (price[i-4]*5+price[i-3]*4+price[i-2]*3+price[i-1]*2+price[i]*1)/15
        WMAlist.append(round(WMA,2))
    
    HMA = []
    for i in range(len(WMAlist)):
        HMA += [round(math.sqrt(WMAlist[round(i/2)] * 2 - WMAlist[i])) ]
    

    print(HMA)
    return HMA

#     show(p)


############ Function Call ############
insight5("GOOGL")


[54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54,

[54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,


### Insight 5 Explanation

A Hull Moving Average or HMA was initially created to overcome the lag that was present when calculating Simple Moving Averages. In fact both the exponential and Weighted Moving Avergaes were developed to minimize that lag. The Hull Moving Average (HMA), developed by Alan Hull, is an extremely fast and smooth moving average. In fact, the HMA almost eliminates lag altogether and manages to improve smoothing at the same time. (Fidelity)

## Data Visualizations

In [402]:
def visual1():
  pass





############ Function Call ############
insight1("GOOGL")

### Visualization 1 Explanation

This is the OBV indicator graph and the mentioned stock's close graph. Refer to insight 1 for more info on OBV. Insert any stock's ticker/symbol as a string in the funtion call above to see it's movement and the indicator. 
Note: If you see divergence bwteen graphs, that signals a reversal.

In [418]:
def visual2():
  pass





############ Function Call ############
insight2("GOOGL")

### Visualization 2 Explanation

This is the VPT indicator graph and the mentioned stock's close graph. Refer to insight 2 for more info on VPT.Insert any stock's ticker/symbol as a string in the funtion call above to see it's movement and the indicator. 
Note: If you see divergence bwteen graphs, that signals a reversal.

In [368]:
def visual3():
  pass





############ Function Call ############
insight3("GOOGL")

### Visualization 3 Explanation
<b> Hover over bars to see values </b> <br>
This is an interactive bar chart for the mentioned stock's volatilties. Refer to insight 3 for more info on volatilities. Insert any stock's ticker/symbol as a string in the funtion call above to see it's volatilites. 
Note: If you see a lower IV than HV that means a stock may gain market's interest soon and might be a potential investment.

## Summary File

All code here is robust and versatile for all stocks in the NASDAQ stock exchange. For insight1 and insight2 the data is available for all listed stocks in the NASDAQ and NYSE exchange. Insight3 contains weekly-update historical and implied volatility data for futures and options available as well if that is desired.<br>
This summary code can be used for any stock, but for simplicity's sake I am using Google' stock (GOOGL) for anaylsis. This will summarize if the google stock is a buy or a sell. Data from insights 1 and 2 is not updated in real time, so this conclusion should not be considered accurate by any means or used in making financial decisions. Data from insights 3-5 is updated in real time so can be a useful tool in making decisions. This Conclusion will be based on 5 ways of analysis I have worked with above. These are the OBV indicator, the VPT indicator, Historical/Implied Volatility, the Weighted Moving Average, and the Hull Moving Average. 

In [470]:
from pprint import pprint
import pandas as pd
import pandas_datareader as pdr
import datetime as dt

def summary1(stock):
    obv = insight1(stock)
    vpt = insight2(stock)
    hviv = insight3(stock)
    wma = insight4(stock)
    hma = insight5(stock)
    obvdf = pd.DataFrame(obv,columns=["OBV Value"])
    vptdf = pd.DataFrame(vpt,columns=["VPT Value"])
    columns=['20-day HV','50-day HV','100-day HV','Average HV','IV']
    wmadf = pd.DataFrame(wma,columns=["WMA value"])
    
    
#     pprint(df)
    start = dt.datetime(2021, 9, 1)
    end = dt.datetime.now()
    ticker = pdr.get_data_yahoo(stock, start, end)['Adj Close']
    frames = [wmadf,obvdf,vptdf,ticker]
    df = pd.concat(frames, axis = 1)
    
    df.to_csv("D:/Downloads/summary.csv", index = True)
    
#     Since we see the indicators dont diverge against the stock trendline. It is considered a buy signal for Google.
    
    
    


############ Function Call ############
summary1("GOOGL")

# Cited Sources

If you used any additional sources to complete your Data Analysis section, list them here:


*   Example Module Documentation
*   Example Stack Overflow Assistance

https://corporatefinanceinstitute.com/resources/knowledge/trading-investing/on-balance-volume-indicator-obv/

https://corporatefinanceinstitute.com/resources/knowledge/trading-investing/volume-price-trend-indicator-vpt/

https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/wma

https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/hull-moving-average



# Video Presentation

If you uploaded your Video Presentation to Bluejeans, YouTube, or any other streaming services, please provide the link here:


*   Video Presentation Link



Make sure the video sharing permissions are accessible for anyone with the provided link.

# Submission

Prior to submitting your notebook to Gradescope, be sure to <b>run all functions within this file</b>. We will not run your functions ourselves, so we must see your outputs within this file in order to receive full credit.
